# Mediods Analysis

This experiment has the purpose of analysing the medoid of some of the clusters defined in the previous experiment. According to it, the best number of cluster is 32 according with elbow method.

In [1]:
#!pip install numpy==1.15 wordcloud

In [2]:
import sys
import os
sys.path.append('../')
from src import reader as r
from src import visualization as v

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
%matplotlib notebook
import numpy as np
import pandas as pd
import sklearn
print(sklearn.__version__)
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import pairwise_distances

0.20.0


In [4]:
X = r.readWord2Vec()
npX = X
print(X[:10])
print(X.shape)

['word2vec.csv', 'bags.csv', 'health.txt', 'health-dataset.zip', 'health-dataset']
[[ 0.02987077 -0.15110606 -0.02884087 ...  0.02446168 -0.08834651
  -0.09221231]
 [ 0.05298314 -0.05420527  0.02592565 ...  0.01782615 -0.02950471
   0.00508323]
 [ 0.07016749 -0.05757345 -0.13483836 ...  0.10909334 -0.0250241
  -0.0654501 ]
 ...
 [ 0.10532002 -0.05241808 -0.02433    ... -0.01405231  0.03333547
   0.01318201]
 [ 0.10429937 -0.1797766  -0.05073992 ...  0.01325834 -0.18105656
  -0.07903843]
 [ 0.13153867 -0.05729359 -0.04480435 ... -0.00396843 -0.05132721
   0.00955163]]
(13229, 128)


In [5]:
from importlib import reload
reload(r)
news = r.readNews()
print(news.head(10))
print(news.shape)

['word2vec.csv', 'bags.csv', 'health.txt', 'health-dataset.zip', 'health-dataset']
Read 13229 lines
                   id                    publish_date  \
0  576880531301801984  Sat Mar 14 23:00:11 +0000 2015   
1  576820122666471424  Sat Mar 14 19:00:08 +0000 2015   
2  576744652717461504  Sat Mar 14 14:00:15 +0000 2015   
3  576736754436304896  Sat Mar 14 13:28:52 +0000 2015   
4  576736614766010368  Sat Mar 14 13:28:18 +0000 2015   
5  576548368740052992  Sat Mar 14 01:00:17 +0000 2015   
6  576518190286536704  Fri Mar 13 23:00:22 +0000 2015   
7  576494177971732480  Fri Mar 13 21:24:57 +0000 2015   
8  576472874946433024  Fri Mar 13 20:00:18 +0000 2015   
9  576464606551490560  Fri Mar 13 19:27:27 +0000 2015   

                                       headline_test  
0  An abundance of online info can turn us into e...  
1  A plant-based diet that incorporates fish may ...  
2  It doesn't take much to damage your hearing at...  
3  RT @CNN: Forever young? Discover this island’s...

In [6]:
from MulticoreTSNE import MulticoreTSNE as TSNE

smp_sz = X.size
tsne_bow = TSNE(n_components=2, perplexity=10, verbose=True, n_jobs=-1)#500
tsne_bow_result = tsne_bow.fit_transform(X)

## Computing the Medoids
In here, we compute the medoids as the closest point to the centroids of the clusters.

In [7]:
best_K = 32
print("#############################")
print("Best K =", best_K)
print("Applying K-means")
best_cluster = KMeans(n_clusters=best_K, n_jobs=-1)
best_cluster_result = best_cluster.fit(X)
print("Finished")
print("#############################")    

#############################
Best K = 32
Applying K-means
Finished
#############################


In [8]:
y_pred = best_cluster_result.labels_
arrays=[]
for i in range(best_K):
    dist = np.linalg.norm(best_cluster_result.cluster_centers_[i]-npX[y_pred==i],axis=1)
    index = np.argmin(dist)
    arrays.append(npX[y_pred==i][index])   

medoids = np.vstack(arrays)
print(medoids.shape)
print(medoids)

(32, 128)
[[ 0.12332614  0.01960983 -0.0883598  ...  0.08688914 -0.10242594
  -0.01897467]
 [ 0.11083259 -0.10522303  0.03524468 ...  0.03864767  0.00544055
  -0.06069441]
 [ 0.14511045 -0.1618841  -0.1077574  ...  0.0495424  -0.03909925
  -0.12398614]
 ...
 [ 0.11508613 -0.1850927  -0.03711164 ...  0.05180415 -0.05892295
  -0.05880312]
 [ 0.11582299 -0.09536775 -0.01162747 ...  0.03790362  0.01347604
  -0.0544413 ]
 [ 0.16753181  0.03649224 -0.03392395 ...  0.01264674 -0.04756675
  -0.05116208]]


In [9]:
l = []
for i in range(best_K):
    l.append(np.where(np.all(npX==medoids[i],axis=1))[0][0]) # these last [0][0] return the index, since the output of this command is (array([id]),)
    
medoids_index = np.array(l)
print(medoids_index)
print(len(np.unique(medoids_index)))

[12187  1049  7064  6889  8131  2381  3153  6046   610  7194  7631 10253
   474  7898   493  8773  6089  7411  8672   741  7341  7725  2897  4309
  1999  7987  7717  1115  1386  8664  1810  5716]
32


## Selecting medoids neighbors
Here the medoids neighbors are selected in order to plot them and see if the clusters make sense.

In [10]:
nm = 5 #number of medoids to be analyzed
nn = 5 #number of neighbor of each medoids
np.random.seed(42)
selected_medoids_index = np.random.randint(0,best_K,nm)
selected_medoids = medoids[selected_medoids_index]
print("Random indexes:",selected_medoids_index)
indexes_per_medoid = []

for i in range(nm):
    p = selected_medoids[i]
    aux = npX[y_pred==selected_medoids_index[i]]
    d = np.linalg.norm(aux-p,axis=1)
    points = []
    points.append(medoids_index[selected_medoids_index[i]])
    
    for j in range(nn):
        min_d = d.argmin()
        while (d[min_d] == 0.):
            d[min_d] = np.inf
            min_d = d.argmin()
        points = points + (np.where(np.all(npX==aux[min_d],axis=1))[0]).tolist()
        d[min_d] = np.inf
        
    indexes_per_medoid.append(np.unique(points).tolist())

    
print("Points:",indexes_per_medoid)
print(len(indexes_per_medoid))

Random indexes: [ 6 19 28 14 10]
Points: [[567, 1135, 1714, 3153, 7038, 8434], [33, 576, 741, 895, 1075, 10909], [806, 1313, 1386, 3295, 7892, 8235], [493, 1730, 2173, 3385, 3506, 7944, 7977], [6564, 6846, 7608, 7631, 7931, 8346]]
5


## Analyzing the Medoids
Here we start the analysis of the medoids and their neighbors.

### Exporting the news of the medoids and clusters

In [12]:
import os 

os.makedirs('../output/w2vec_clusters/', exist_ok=True)
os.makedirs('../output/w2vec_medoids/', exist_ok=True)

print("Random indexes:",selected_medoids_index)
for i in range(best_K):
    np.savetxt("../output/w2vec_clusters/k_"+str(i)+".txt", news[y_pred==i].values, fmt='%s', encoding='utf8')
for i in range(selected_medoids_index.size):
    p = indexes_per_medoid[i]
    np.savetxt("../output/w2vec_medoids/k_"+str(selected_medoids_index[i])+".txt", np.take(news,p,0).values, fmt='%s')

Random indexes: [ 6 19 28 14 10]


### News of the medoids and their neighbors

In [13]:
for p in indexes_per_medoid:
    info = np.take(news,p,0).loc[:,"headline_test"].values
    print("----------------------------")
    for i in range(info.size):
        print(info[i],"\n")
print("----------------------------")

----------------------------
When you're sick, the last thing you want to do is get out of bed to see a doctor. These apps bring the doc to you! 

Do you have #autism or a loved one on the ASD spectrum? How do you communicate? Share your story w/ @cnnireport 

RT @PresJrCouncil: @deebeachgirl @cnnhealth Totally worth it! "You are what you eat. What do you want to be?" #betterinbalance 

RT @jdwilson2: Do you have a favorite T-shirt you just can't give up? Share a pic &amp; what it says about you using #TshirtTales 

If you want to tone up and lose weight you will love this fast-paced fat shedding workout: 

You just can't help but notice her awesome physique! Jennifer Aniston's diet and fitness secrets: 

----------------------------
RT @CNNMoney: Tyson shares dropped after officials confirmed cases of bird flu in the U.S. 

#Ebola fears hit close to home: American Patrick Sawyer died in Nigeria after being infected by the deadly virus 

CDC worries mosquito-borne virus could hit U.S. 

### Generating cloud of words

In [14]:
%matplotlib inline
from wordcloud import WordCloud
import matplotlib
import matplotlib.pyplot as plt

os.makedirs('../output/w2vec_wordcloud_clusters/', exist_ok=True)
os.makedirs('../output/w2vec_wordcloud_medoids/', exist_ok=True)

# clusters
for i in range(best_K):
    info = ' '.join(news[y_pred==i].loc[:,"headline_test"])
    wordcloud = WordCloud(random_state=42).generate(info)
    fig = plt.figure()
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")    
    fig.savefig("../output/w2vec_wordcloud_clusters/k_"+str(i))
    plt.close(fig)

#medoids
for i in range(selected_medoids_index.size):
    p = indexes_per_medoid[i]
    info = ' '.join(np.take(news,p,0).loc[:,"headline_test"])
    wordcloud = WordCloud(random_state=42).generate(info)
    fig = plt.figure()
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")    
    fig.savefig("../output/w2vec_wordcloud_medoids/k_"+str(selected_medoids_index[i]))
    plt.close(fig)
